In [10]:
import pandas as pd
import numpy as np
import math
import datetime
from ipynb.fs.defs.functions import check_cities
table = pd.read_csv("../Dati/orari/route.csv", delimiter=";")
time_table = pd.read_csv("../Dati/orari/route_time.csv", delimiter=";")
dft = pd.read_csv("../Dati/distanze/times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
dfd = pd.read_csv("../Dati/distanze/distances_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
#fur1 = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";")
#fur2 = pd.read_csv("../Dati/turni_f/furgoncino2_routing.csv", delimiter=";")

def get_row(df, string):
    return pd.Series(df.index).str.contains(string, case=False)

def get_column(df, string):
    return pd.Series(df.columns).str.contains(string, case=False)

def difference(day, index, index2):
    time = datetime.datetime.strptime(time_table[day][index], '%H:%M')
    time2 = datetime.datetime.strptime(time_table[day][index2], '%H:%M')
    return ((time2-time).seconds)/60

#FIXED tiene conto delle pause pranzo
def difference2(day, index, index2):
    time = datetime.datetime.strptime(time_table[day][index], '%H:%M')
    time2 = datetime.datetime.strptime(time_table[day][index2], '%H:%M')
    
    if(time > datetime.datetime.strptime('12:00', '%H:%M') and 
       time < datetime.datetime.strptime('13:25', '%H:%M') and
       ((time2-time).seconds)/60 > 30):
        time = time + datetime.timedelta(hours=1)
        #print(time)
    return ((time2-time).seconds)/60

In [2]:
def compute(schedule, df):
    row = df[get_row(df, schedule[0].strip()).values]
    tot = 0
    #print(str(schedule[0]) + " " + str(tot))
    for i in range(1, len(schedule.dropna())):
        try:
            val = row.loc[:, get_column(df, schedule[i].strip()).values].values[0][0]
        except:
            print(row.index)
            print(schedule[i])
            raise Exception("Prova")
        #print(str(schedule[i]) + " " + str(val))
        tot += val
        row = df[get_row(df, schedule[i].strip()).values]
    return round(tot, 2)

def addHeadTail(schedule):
    formatted = list(schedule.dropna())[:]
    formatted.insert(0, "Cremona, Sesto 39")
    formatted.append("Ostiano, Mazzini, 1")
    return formatted

In [3]:
week = ["lunedi", "martedi", "mercoledi", "giovedi", "venerdi"]
fur1_time_greedy = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";", encoding = "ISO-8859-1")
fur1_time_local = pd.read_csv("../Dati/turni_f/ls1_shifts.csv", delimiter=";", encoding = "ISO-8859-1")
fur1_time_2opt = pd.read_csv("../Dati/turni_f/2-opt1_shifts.csv", delimiter=";", encoding = "ISO-8859-1")
fur1_time_base = pd.read_csv("../Dati/turni_f/f1_original.csv", delimiter=";", encoding = "ISO-8859-1")
fur1_multi = pd.read_csv("../Dati/turni_f/multi_veich_LS1.csv", delimiter=";", encoding = "ISO-8859-1")
print("Comparison 1st pickup with greedy, local search and 2opt solutions (Based on time schedule)")
for day in week:
    greedy_schedule = addHeadTail(fur1_time_greedy[day])
    base_schedule = addHeadTail(fur1_time_base[day])
    print(f"Giorno: {day}  "+
          f"{compute(pd.Series(base_schedule), dft)} "+
          f"{compute(pd.Series(greedy_schedule), dft)} "+
          f"{compute(fur1_time_local[day], dft)} "+
          f"{compute(fur1_time_2opt[day], dft)} min")

Comparison 1st pickup with greedy, local search and 2opt solutions (Based on time schedule)
Giorno: lunedi  224.21 195.12 183.97 178.27 min
Giorno: martedi  222.56 260.85 252.93 216.61 min
Giorno: mercoledi  271.43 163.72 163.72 163.72 min
Giorno: giovedi  239.98 246.81 246.06 223.41 min
Giorno: venerdi  235.36 181.04 181.04 170.61 min


In [4]:
fur1_distance_greedy = pd.read_csv("../Dati/turni_f/furgoncino1_KM_routing.csv", delimiter=";")
fur1_distance_local = pd.read_csv("../Dati/turni_f/ls1_km_shifts.csv", delimiter=";")
fur1_distance_2opt = pd.read_csv("../Dati/turni_f/2-opt1_km_shifts.csv", delimiter=";")
print("Comparison 1st pickup with greedy, local search and 2opt solutions (Based on kilometers schedule)")
for day in week:
    greedy_schedule = addHeadTail(fur1_distance_greedy[day])
    print(f"Giorno: {day}  {compute(pd.Series(greedy_schedule), dfd)}  "+
          f"{compute(pd.Series(fur1_distance_local[day]), dfd)}  "+
          f"{compute(pd.Series(fur1_distance_2opt[day]), dfd)} km")

Comparison 1st pickup with greedy, local search and 2opt solutions (Based on kilometers schedule)
Giorno: lunedi  189.52  185.86  161.94 km
Giorno: martedi  314.19  307.48  296.46 km
Giorno: mercoledi  170.99  163.62  155.37 km
Giorno: giovedi  237.83  225.69  186.6 km
Giorno: venerdi  162.72  162.47  159.14 km


In [5]:
def addHeadTail(schedule):
    formatted = list(schedule.dropna())[:]
    formatted.insert(0, "Cremona, Sesto 39")
    formatted.append("San Giovanni in Croce, Feudatari, 25 A")
    return formatted

In [8]:
shortweek = ["martedi", "mercoledi", "giovedi"]
fur2_time_greedy = pd.read_csv("../Dati/turni_f/furgoncino2_routing.csv", delimiter=";", encoding = "ISO-8859-1")
fur2_time_local = pd.read_csv("../Dati/turni_f/ls2_shifts.csv", delimiter=";", encoding = "ISO-8859-1")
fur2_time_2opt = pd.read_csv("../Dati/turni_f/2-opt2_shifts.csv", delimiter=";", encoding = "ISO-8859-1")
fur2_time_base = pd.read_csv("../Dati/turni_f/f2_original.csv", delimiter=";", encoding = "ISO-8859-1")

print("Comparison 2nd pickup with greedy, local search and 2opt solutions (Based on time schedule)")
for day in shortweek:
    greedy_schedule = addHeadTail(fur2_time_greedy[day])
    base_schedule = addHeadTail(fur1_time_base[day])
    print(f"Giorno: {day}  " +
          f"{compute(pd.Series(base_schedule), dft)}  "+
          f"{compute(pd.Series(greedy_schedule), dft)}  "+
          f"{compute(fur2_time_local[day], dft)}  "+
          f"{compute(fur2_time_2opt[day], dft)} min")

Comparison 2nd pickup with greedy, local search and 2opt solutions (Based on time schedule)
Giorno: martedi  201.86  298.71  292.23  237.3 min
Giorno: mercoledi  275.74  143.73  143.73  135.54 min
Index(['Cingia de' Botti, Giuseppina, 79'], dtype='object')
Martignana di Po, LibertÃ , 64


Exception: Prova

In [7]:
fur2_distance_greedy = pd.read_csv("../Dati/turni_f/furgoncino2_KM_routing.csv", delimiter=";")
fur2_distance_local = pd.read_csv("../Dati/turni_f/ls2_km_shifts.csv", delimiter=";")
fur2_distance_2opt = pd.read_csv("../Dati/turni_f/2-opt2_km_shifts.csv", delimiter=";")
print("Comparison 2nd pickup with greedy, local search and 2opt solutions (Based on kilometers schedule)")
for day in shortweek:
    greedy_schedule = addHeadTail(fur2_distance_greedy[day])
    print(f"Giorno: {day}  {compute(pd.Series(greedy_schedule), dfd)}  "+
          f"{compute(pd.Series(fur2_distance_local[day]), dfd)}  "+
          f"{compute(pd.Series(fur2_distance_2opt[day]), dfd)} km")

Comparison 2nd pickup with greedy, local search and 2opt solutions (Based on kilometers schedule)
Giorno: martedi  108.93  108.85  108.67 km
Giorno: mercoledi  164.2  162.67  162.67 km
Giorno: giovedi  236.13  236.13  173.54 km


In [9]:
def compute_week(days, lista, add):
    res = 0
    for day in days:
        if add:
            res += compute(pd.Series(addHeadTail(lista[day])), dft)
        else:
            res += compute(lista[day], dft)
    return res

def addHeadTail(schedule):
    formatted = list(schedule.dropna())[:]
    formatted.insert(0, "Cremona, Sesto 39")
    formatted.append("Ostiano, Mazzini, 1")
    return formatted

In [10]:
fur1_time_greedy = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";", encoding = "ISO-8859-1")
fur1_time_local = pd.read_csv("../Dati/turni_f/ls1_shifts.csv", delimiter=";", encoding = "ISO-8859-1")
fur1_time_2opt = pd.read_csv("../Dati/turni_f/2-opt1_shifts.csv", delimiter=";", encoding = "ISO-8859-1")
fur1_time_base = pd.read_csv("../Dati/turni_f/f1_original.csv", delimiter=";", encoding = "ISO-8859-1")
fur1_multi = pd.read_csv("../Dati/turni_f/multi_veich_LS1.csv", delimiter=";", encoding = "ISO-8859-1")

print("Greedy", round(compute_week(week, fur1_time_greedy, True),2), sep=": ")
print("Time base", round(compute_week(week, fur1_time_base, True),2), sep=": ")
print("Local search", round(compute_week(week, fur1_time_local, False),2), sep=": ")
print("2opt", round(compute_week(week, fur1_time_2opt, False),2), sep=": ")
print("Multi veichle", round(compute_week(week, fur1_multi, False),2), sep=": ")

Greedy: 1047.54
Time base: 1193.54
Local search: 1027.72
2opt: 952.62
Multi veichle: 572.55


In [11]:
def addHeadTail(schedule):
    formatted = list(schedule.dropna())[:]
    formatted.insert(0, "Cremona, Sesto 39")
    formatted.append("San Giovanni in Croce, Feudatari, 25 A")
    return formatted

In [12]:
fur2_time_greedy = pd.read_csv("../Dati/turni_f/furgoncino2_routing.csv", delimiter=";")
fur2_time_local = pd.read_csv("../Dati/turni_f/ls2_shifts.csv", delimiter=";")
fur2_time_2opt = pd.read_csv("../Dati/turni_f/2-opt2_shifts.csv", delimiter=";", encoding = "ISO-8859-1")
fur2_time_base = pd.read_csv("../Dati/turni_f/f2_original.csv", delimiter=";", encoding = "ISO-8859-1")
fur2_multi = pd.read_csv("../Dati/turni_f/multi_veich_LS2.csv", delimiter=";", encoding = "ISO-8859-1")
shortweek=["martedi", "mercoledi", "giovedi"]
print("Greedy", round(compute_week(shortweek, fur2_time_greedy, True),2), sep=": ")
print("Time base", round(compute_week(shortweek, fur2_time_base, True),2), sep=": ")
print("Local search", round(compute_week(shortweek, fur2_time_local, False),2), sep=": ")
print("2opt", round(compute_week(shortweek, fur2_time_2opt, False),2), sep=": ")
print("Multi veichle", round(compute_week(shortweek, fur2_multi, False),2), sep=": ")

Greedy: 601.22
Time base: 455.06
Local search: 544.69
2opt: 481.57
Multi veichle: 426.91


In [16]:
week = ["lunedi", "martedi", "mercoledi", "giovedi", "venerdi"]
short_week = ["martedi", "mercoledi", "giovedi"]


fur1_local = pd.read_csv("../Dati/turni_f/ls1_shifts.csv", delimiter=";", encoding = "ISO-8859-1")
fur1_2opt = pd.read_csv("../Dati/turni_f/2-opt1_shifts.csv", delimiter=";", encoding = "ISO-8859-1")
fur1_multi = pd.read_csv("../Dati/turni_f/multi_veich_LS1.csv", delimiter=";", encoding = "ISO-8859-1")

fur2_local = pd.read_csv("../Dati/turni_f/ls2_shifts.csv", delimiter=";", encoding = "ISO-8859-1")
fur2_2opt = pd.read_csv("../Dati/turni_f/2-opt2_shifts.csv", delimiter=";", encoding = "ISO-8859-1")
fur2_multi = pd.read_csv("../Dati/turni_f/multi_veich_LS2.csv", delimiter=";", encoding = "ISO-8859-1")

oss_ls_1 = pd.read_csv("../Dati/turni_f/optimized_start/oss_ls1.csv", delimiter=";", encoding = "ISO-8859-1")
oss_ls_2 = pd.read_csv("../Dati/turni_f/optimized_start/oss_ls2.csv", delimiter=";", encoding = "ISO-8859-1")

oss_2opt_1 = pd.read_csv("../Dati/turni_f/optimized_start/oss_2opt_f1.csv", delimiter=";", encoding = "ISO-8859-1")
oss_2opt_2 = pd.read_csv("../Dati/turni_f/optimized_start/oss_2opt_f2.csv", delimiter=";", encoding = "ISO-8859-1")

oss_mvls_1 = pd.read_csv("../Dati/turni_f/optimized_start/oss_MVLS_f1.csv", delimiter=";", encoding = "ISO-8859-1")
oss_mvls_2 = pd.read_csv("../Dati/turni_f/optimized_start/oss_MVLS_f2.csv", delimiter=";", encoding = "ISO-8859-1")
#print('fur1')
oss_ls1 = 0
oss_opt1 = 0
oss_mvls1 = 0
ls1 = 0
opt1 = 0
mvls1 = 0
for day in week:
    #greedy_schedule = addHeadTail(fur1_time_greedy[day])
    #base_schedule = addHeadTail(fur1_time_base[day])
    oss_ls1 = oss_ls1 + compute(oss_ls_1[day], dft)
    oss_opt1 = oss_opt1 + compute(oss_2opt_1[day], dft)
    oss_mvls1 = oss_mvls1 + compute(oss_mvls_1[day], dft)
    
    ls1 = ls1 + compute(fur1_local[day], dft)
    opt1 = opt1 + compute(fur1_2opt[day], dft)
    mvls1 = mvls1 + compute(fur1_multi[day], dft)
#print('\noss\n')  
#print(f"ls = {round(oss_ls, 2)} "+
#      f"2opt = {round(oss_opt, 2)} ")
#print('\nnon ottimizzato\n')
#print(f"ls = {round(ls, 2)} "+
#      f"2opt = {round(opt, 2)} ")


#print('\n') 
#print('fur2')
oss_ls2 = 0
oss_opt2 = 0
oss_mvls2 = 0
ls2 = 0
opt2 = 0
mvls2 = 0
for day in short_week:
    #greedy_schedule = addHeadTail(fur1_time_greedy[day])
    #base_schedule = addHeadTail(fur1_time_base[day])
    oss_ls2 = oss_ls2 + compute(oss_ls_2[day], dft)
    oss_opt2 = oss_opt2 + compute(oss_2opt_2[day], dft)
    oss_mvls2 = oss_mvls2 + compute(oss_mvls_2[day], dft)
    
    ls2 = ls2 + compute(fur2_local[day], dft)
    opt2 = opt2 + compute(fur2_2opt[day], dft)
    mvls2 = mvls2 + compute(fur2_multi[day], dft)
    
#print('\noss\n')
#print(f"ls = {round(oss_ls, 2)} "+
#      f"2opt = {round(oss_opt, 2)} ")
#print('\nnon ottimizzato\n')
#print(f"ls = {round(ls, 2)} "+
#      f"2opt = {round(opt, 2)} ")
#print('\n')
print(f"oss_ls = {round(oss_ls1 + oss_ls2, 2)} " + 
      f"ls = {round(ls1 + ls2, 2)}")
print(f"oss_opt = {round(oss_opt1 + oss_opt2, 2)} " + 
      f"opt = {round(opt1 + opt2, 2)}")
print(f"oss_mvls = {oss_mvls1 + oss_mvls2} " + 
      f"mvls = {round(mvls1 + mvls2, 2)}")

oss_ls = 1657.99 ls = 1572.88
oss_opt = 1485.5 opt = 1434.66
oss_mvls = 1158.09 mvls = 1241.55


In [11]:
print(f"diff_ls = {check_cities(fur1_local, fur2_local, oss_ls_1, oss_ls_2)}")
print(f"diff_2opt = {check_cities(fur1_2opt, fur2_2opt, oss_2opt_1, oss_2opt_2)}")
print(f"diff_2opt = {check_cities(fur1_multi, fur2_multi, oss_mvls_1, oss_mvls_2)}")

diff_ls = []
diff_2opt = []
diff_2opt = []
